# 7. 장고로 앱 만들기
모든 장고 프로젝트는 1개 이상의 앱으로 구성된다. 이 때  '앱'이란 '특정한 기능을 수행하는 단위 모듈'로 생각하면된다.  
예를 들어 블로그와 갤러리, 방명록의 3가지 기능을 갖는 웹사이트를 만들 때 일반적으로 3개의 앱을 만들어 개발하고 관리한다. 물론 앱의 개수는 상황에 따라 다르지만 이 프로젝트에서는 2개의 앱을 만들것이다. 하나는 blog앱이며 다른 하나는 대문 페이지와 자기소개 페이지를 보여주기 위한 single_pages앱이다.

___

## blog앱과 single_pages 앱 만들기
### 가상환경 실행 확인 후 blog 앱 만들기
먼저 가상환경이 잘 실행되어 잇는지 확인하자 (터미널 앞에 (venv)가 있으면 된다.)  이제 블로그앱을 만들기 위해 python manage.py startapp blog를 입력해주자.  
같은방식으로 python manage.py startapp single_pages를 입력하여 페이지 앱도 만들어 주자.

___

## 데이터베이스의 개념
웹사이트는 효율적인 정보 관리를 위해 DB를 사용한다.  
앞으로 DB에서 User=작성자 또는 운영자, Post=블로그 글, Comment=댓글로 표기하겠다.

### EXCEL로 DB 이해하기  
#### User 테이블 살펴보기
다음은 User정보를 엑셀 시트에 정리한 것이다.

In [11]:
import pandas as pd
pd.read_excel('example.xlsx',sheet_name=0)

,id,username,password,email,first_name,last_name,is_admin
0,1,a,*********,a@a.com,aaa,AAA,True
1,2,b,*********,b@b.com,bbb,BBB,False
2,3,c,*********,c@c.com,ccc,CCC,False


DB에서는 위와 같은 시트를 테이블 이라고 한다. 가로 방향(행)으로 읽는 데이터를 레코드 라고 하며, 세로방향(열)로 읽는 데이터는 '필드'라고 한다.  
DB용어로 읽어보면 id, username, password, email, first_name, last_name, is_admin 필드가 있고, 총 3개의 레코드가 있다. 이때 장고에서는 레코드에 숫자로 된 id를 자동으로 부여하며 id는 고유할 필드이므로 기본키라고 부른다.

#### Post 테이블 살펴보기

In [12]:
pd.read_excel('example.xlsx',sheet_name=1)

,id,title,author,created_at,content
0,1,장고는 왜 좋을까,2,2020-10-29,장고는 파이썬 개바랒의 시간과 노력을 줄여준다.
1,2,파이썬을 배워야 하는 이유,1,2020-10-30,파이썬은 배우기 쉬우며 확장성이크다.


Post 테이블은 title(제목), author(작성자), created_at(작성일), content(본문 내용)의 필드를 가진다.  
그런데 author 필드는 특이하게 숫자로 되어있따. 첫번 째 레코드의 author 필드는 2이다. 이것은 User 테이블의 id가 2인 레코드르 의미한다.  
이런식으로 테이블과 테이블이 연결되며 이렇게 테이블이 다른 테이블의 레코드를 id로 지정하는 방법을 외래키(foreign key)라고 한다.

#### Comment 테이블 살펴보기

In [17]:
pd.read_excel('example.xlsx',sheet_name=2)

,id,post,author,created_at,content
0,1,1,1,2021-10-29,플라스크에 비해 나은점은 뭔가요?
1,2,1,2,2021-10-30,둘다 장담점이 있다.
2,3,2,3,2021-10-31,좋은글 입니다.


Comment 테이블은 post,author 2개의 외래키를 가진다.  
첫번 째 레코드의 post, author필드를 보면 title이 '장고는 왜 좋은가'인 post레코드와 username이 'a'인 User레코드에 연결된 댓글임을 알 수 있다.(id가 1) 즉, 이 댓글은 '장고는 왜 좋은가' 라는 제목의 포스트에 'a'라는 사용자가 남긴 댓글이다. 이를 통해  Comment 테이블은 Post, User 테이블에 연결되어 있음을 자연스럽게 이해할 수 있다.

___

## 모델 만들기
장고의 장점 중 하나는 모델을 이용해 장고 웹 프레잌워크 안에서 DB를 관리할 수 있다는 것이다. 모델은 데이터를 저장하기 위한 하나이 단위라고 보면된다. 일반적으로 DB를 다루려면 SQL등의 언어를 또 배워야 하는데, 이는 입문자에게는 진입장벽이 될 수 있따. 장고의 모델을 이요하면 파이썬만으로 CRUD기능을 쉽게 구현할 수 있을 뿐만 아니라 관리자 페이지, 입력 폼등도 쉽게 만들 수 있다.
### 블로그의 글을 위한 모델만들기
#### (1) Post모델 만들기
블로그의 핵심인 포스트의 형태를 정의하는 Post모델을 만들어보자. 포스트에는 어떤 정보가 필요할까?  
제목(title), 내용(content), 작성일(created_at), 작성자 정보(author)가 필요하다. blog/model.py를 열어 다음과같이 입력한다.

In [25]:
# from django.db import models  

# class Post(models.Model):  
#     title=models.CharField(max_length=30)  
#     content=models.TextField()  

#     created_at=models.DateTimeField()  
    # autor : 미정

위 코드를 보면 알 수 있듯이 Post 모델은 models 모듈이 Model클래스를 확장해서 만든 파이썬 클래스이다. 앞으로 대부분의 모델은 이런 방식으로 만든다.  
- title 필드는 CharField(작거나 중간 크기의 고정된 길이의 문자열을 정의할 때 사용)클래스로 만들고 최대길이는 30으로 설정한다..  
- conntent필드는 문자열 길이 제한이 없는 TextField를 사용해 만든다.  
- created_at 필드는 DateTimeField로 만든다. DateTimeField는 월,일,시,분,초 까지 기록할 수있게 해주는 필드를 만들때 사용한다.  
- autor필드는 외래키를 구현할 때 다룰 것이므로 이정도만 이해한다.  
#### (2) DB에 Post 모델 반영하기
아직 Post 모델은 파이썬 클래스로만 존재한다. 이를 DB에 반영해야 실제 테이블이 생서된다.  
터미널에 python manage.py makemigrations를 입력한다. 그럼 No changes detected라고 뜨는데 아직 blog 앱을 등록하지 않았기 때문이다.
#### (3) settings.py에 blog 앱 등록하기
settings.py에는 INSTALLED_APPS라는 리스트가 있다. 여기에 'blog','single_pages' 앱을 추가한다.
#### (4) DB에 Post 모델 반영하기
이제 python manage.py makemigrations를 다시 입력한다. 그러면 blog앱에 변화가 있음을 메시지로 알려주고 blog/migirations 폴더에 0001_initial.py파일이 생성된다. 이 상태는 아직 DB모델에 적용되지 않은 상태이므로 실제 DB에 적용하려면 python manage.py migrate명령을 실행해야 한다. 0001_initial.py이 생겼고 이 파일은 최초  
makemigrations 명령 실행 시 생기는 파일이다. 그런데 이 파일은 버전관리 대상이 아니므로 깃 관리 대상에서 제외한다.
#### (5) .gitignore 수정하기
gitignore에서 migrations/를 추가한다.
> 관리하지 않는 이유  
> 앞으로 블로그를 개발하다 보면 models.py에 수정을 자주해야한다. 그리고 최종 결과물만 서버에 적용되는데 모델 수정내역을 일일이 기록하다보면 로컬 컴퓨터의 DB와 서버의 DB가 일치 하지 않는 문제가 생길 수 있다.

### 관리자 페이지에서 첫 포스트 작성
관리자 페이지를 들어가면 무엇이 바뀌었는지 파악할 수 있다. python manage.py runserver를 입력해 서버를 실행하고 http://127.0.0.1:8000/admin/ 으로 접속해서 관리자 페이지를 연다.
#### (1) admin.py에 Post모델 추가하기
blog/admin.py 파일을 열고 아래 두줄을 추가해 관리자 페이지에 Post모델을 등록한다.

from .models import Post  
admin.site.register(Post)  

다시 관리자 페이지를 열면 BLOG 라는 섹션이 생겼고 그 아래엔 Posts라는 메뉴가 생긴것을 알 수있다.
#### (2) 새로운 포스트 생성하기
Posts 메뉴를 클릭하면 아직은 작성한 포스트가 없어 빈 페이지가 뜨지만 add버튼을 누르면 포스트를 생성할 수 있는 페이지가 열린다.  앞서 models.py에서 CharField로  
지정했던 title, TextField로 지정했던 content, DateTimeField로 지정했던 created_at 필드에 값을 넣을 수 있게 각각 양식에 맞춰 입력란이 만들어져 있다. 실제 블로그에 글을 쓰듯 원하는 내용을 채운뒤 SAVE를 누르면 포스트가 생성된다.

### 포스트 개선하기
#### (1) __str__()함수로 포스트 제목과 번호 보 여주기
포스트를 하나 더 생성해보면 포스트가 쭉 나열되지만 포스트 제목이 나타나지 않는다. 이문제는 Post 모델에서 __str__()함수를 선언하면된다. blog/models.py에서 다음 내용을 추가한다.  

def __str__(self):  
     return f'[{self.pk}]{self.title}'  
     
self.pk는 해당 포스트의 pk 값이며 / self.title은 해당 포스트의 title 값이다.  
장고의 모델을 만들면 기본적으로 pk필드가 만들어진다. pk는 각 레코드에 대한 고유값이라고 이해하면 된다. 포스트를 처음 생성하면 자동으로 pk에는 1이 부여되고 두번 째는 2가 부여된다. 이 값을 이용해서 포스트의 제목과 번호를 문자열로 표현한다. 그럼 다음과같이 포스트의 제목이 나타나는 것을 볼 수 있다.

	[2]두번 째 포스트  
	[1]첫번 째 포스트  
#### (2) 특정 지역 기준으로 작성 시간 설정하기
Created at에서 아래를 보면 'Note: You are 9 hours ahead of sever time' 이라는 문구가 나타나며 실제로 9시간의 차이가 난다. 이건 그리니치 표준시에 맞춘 시각 이기 때문이다. 만약 서울을 기준으로 작성시간을 설정하고 싶다면 setting.py에 들어가 TIME_ZONE='UTC' 라고 되어 있는 부분을 Asia/Seoul로 수정한뒤 USE_TZ를 False로 설정한다.
#### (3) 자동으로 작성 시각과 수정시각 저장하기
time버튼을 누르지않고 포스트를 작성하면 자동으로 시간이 저장되면 더욱 편리할것이다. 수정 할 때 역시 마찬가지이다. 수정 시각을 저장할 updated_at 필드를 DateTimeField로 하나 더 만들면 된다. DateTimeField는 auto_now와 auto_now_add라는 설정이 있어 처음 레코드가 생성된 시점, 마지막으로 저장된 시점을 자동으로 저장할 수 있다.  

우선 created_at 필드에 auto_now_add=True로 설정해 처음 레코드가 생성될 때 현재 시각이 자동으로 저장되게 한다. 그 다음 updated_at 필드를 만들고 auto_now=True로 저장하여 다시 저장할 때마다 그 시각이 저장 되게 한다.  

모델을 수정하였다면 이를 makemigrations으로 장고에게 알려주고, migrate로 DB에 반영을 해주자.
다시 관리자 페이지에서 포스트를 작성해보면 더 이상 Created at입력란이 보이지않는다. 저절로 작성시간이 저장되기 때문이다. 다시 저장을 해보면 작성시간과 수정 시각이 자동으로 저장되는것을 확인할 수 있다.


#### 장고 셸 사용하기  
> 파이썬은 스크립트 기반 언어라서 컴파일 과정 없이 한 줄 씩 그때 그때 실행할수 있는 장점이 있다. 이를 활용해 터미널에서 원하는 기능을 간단히 실행시킬 수 있다. 서버를 잠시 중단하고 pyhon manage.py shell을 입력하면 장고 셸이 실행된다. 그럼 아래와 같이 명령어를 사용하여 원하는 정보를 볼 수 있다.  

from blog.models import Post  

p=Post.objects.last()  
p.title  
'두번 째 포스트'  
p.created_at  
datetime.datetime(2021, 10, 31, 7, 18, 42)  
p.updated_at  
datetime.datetime(2021, 10, 31, 16, 30, 21, 317790)  
exit()  